<a href="https://colab.research.google.com/github/youngmoo/ECES-434/blob/main/Class%207.2%20(2021-02-24).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ECES-434: Class 7.2 (2021-02-24)**
Week 7: Project, project, project...!

In [172]:
import numpy as np                      # NumPy, abbreviated as np
import matplotlib.pyplot as plt         # MatplotLib PyPlot module, abbreviated as plt
from matplotlib import animation, rc    # MatplotLib animation module
%matplotlib inline
from scipy import signal                # SciPy's signal module, for DSP functions
import soundfile as sf                  # Switching to the soundfile module for reading and writing soundfiles
import librosa

import IPython.display as ipd           # Interactive Python display module, for playing sounds
from IPython.display import HTML        # For displaying animations
rc('animation', html='jshtml')          # Provides animation controls

ClassPath = '/content/drive/My Drive/ECES-434 Sessions/Class 7-2/'

In [182]:
rc('figure', figsize=(16,4))

In [4]:
# CHANGE THIS to your Drexel username!!
username = 'anonymous'

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Custom plotting functions
Because we're always plotting...

## plotSpectrogram

In [6]:
def plotSpectrogram(sig, fs, win='hann', nseg=512, olap=256, fft_len=512):
  
  f1, t1, Sxx = signal.spectrogram(sig, fs, window=win, nperseg=nseg, noverlap=olap, nfft=fft_len)
  fig = plt.figure(figsize=(16,6))
  
  plt.pcolormesh(t1, f1, 20*np.log10(np.abs(Sxx)))
  plt.ylabel('Frequency (Hz)')
  plt.xlabel('Time (sec)')
  return plt

## myPlot(): properly formats time domain plot of a signal

In [7]:
def myPlot(sig, fs=44100):
  fig = plt.figure(figsize=(16,4))
  t = np.arange(len(sig)) / fs
  plt.plot(t, sig)
  plt.xlabel('Time (sec)')
  return fig, plt

## myPlotFFT(): properly formats frequency domain plot of a signal

In [8]:
def myPlotFFT(sig, n_fft=0, x_lim=22050, fs=44100):
  if n_fft==0:                 
    n_fft = len(sig)                    # Default to length of input signal
  S = np.fft.fft(sig, n_fft)
  N = len(S)
  f = np.arange(N) * fs / N
  fig = plt.figure(figsize=(16,6))
  plt.plot(f, 20*np.log10(np.abs(S)))
  plt.xlim(0, x_lim)
  plt.xlabel('Frequency (Hz)')
  plt.ylabel('Magnitude (dB)')
  return fig, plt  

## myPlotFFTPhase

In [9]:
def myPlotFFTPhase(sig, n_fft=0, x_lim=22050, fs=44100):
  if n_fft==0:                 
    n_fft = len(sig)                    # Default to length of input signal
  S = np.fft.fft(sig, n_fft)
  N = len(S)
  f = np.arange(N) * fs / N
  fig = plt.figure(figsize=(16,4))
  plt.plot(f, np.unwrap(np.angle(S)))
  plt.xlim(0, x_lim)
  plt.xlabel('Frequency (Hz)')
  plt.ylabel('Phase (radians)')
  return fig, plt

## Custom FFT animation functions

In [10]:
n_o = 0
f_size = 2048
n_hop = f_size / 2
N_fft = 4096
fs = 44100
this_f = np.arange(N_fft) * fs / N_fft

# First set up the figure, the axis, and the plot element we want to animate
def setupAnimFFT(x_lim=(0,20000), y_lim=(-120,100)):
  fig = plt.figure(figsize=(14,6))
  ax = plt.axes(xlim=x_lim,ylim=y_lim)
  plt.close()   # Don't output the final figure separately
  line, = ax.plot([], [])
  fig.tight_layout()
  return fig, line

# initialization function: plot the background of each frame
def initAnimFFT():
    line.set_data([], [])
    return (line,)

# animation function. This is called sequentially  
def animateFFT(i, sig):
    n1 = int(n_o + n_hop*i)
    n2 = int(n_o + n_hop*i + f_size)

    x_i = sig[n1:n2]
    X_i = np.fft.fft(x_i * np.hanning(len(x_i)), n=N_fft)
    X_mag = 20*np.log(np.abs(X_i))

    line.set_data(this_f, X_mag)
    return (line,)  

# Usage:
# fig, line = setupAnimFFT()
# anim = animation.FuncAnimation(fig, animateFFT, init_func=initAnimFFT, frames=120, fargs=(signal,), interval=1000/30, blit=True)
# anim

# Continuing with same sound file...
Same as last class: *Need You Tonight* by INXS (1987)

In [ ]:
inxs_s, fs48 = sf.read(ClassPath + 'INXS-48kHz.wav')
inxs = np.mean(inxs_s,axis=1)
print(fs48)
ipd.Audio(inxs,rate=fs48)

# Build a custom "filter" for STFT

In [176]:
nFFT = 2048
nHop = 512

INXS = librosa.stft(inxs,n_fft=nFFT, hop_length=nHop) 
nBins, nFrames = np.shape(INXS)
(nBins, nFrames)

(1025, 2106)

In [ ]:
# Create frequency and time axes
f1 = np.arange(nBins)/(nFFT/2) * (fs48/2)
t1 = np.arange(nFrames) * nHop / fs48

plt.figure(figsize=(16,6))
plt.pcolormesh(t1, f1, 20*np.log10(np.abs(INXS)))
plt.xlabel('Time (sec)')
plt.ylabel('Frequency (Hz)')

# Adjust axis limits
plt.ylim(0,2000)
plt.xlim(0,5)

## Specify your filter range

In [ ]:
plt.plot(f1)
plt.ylim(250,1000)
plt.xlim(10,44)
f1[11:43]

## Design an unevenly weighted triangle

In [ ]:
tri_f = np.zeros(nBins)
tri_f[11:22] = (f1[11:22] - 250)/250
tri_f[22:43] = 1 - (f1[22:43]- 500)/500

plt.plot(f1, tri_f)
plt.xlim(0,4000)

# Save the figure when ready
plt.savefig(ClassPath + 'tri-filter/' + username + '.png')

In [189]:
# Need this in order to tile it for the full STFT matrix
tri_f.shape = (nBins,1)

H = np.tile(tri_f, (1,nFrames)) 
np.shape(H)

(1025, 2106)

In [ ]:
plt.pcolormesh(t1,f1, H)
plt.ylim(0,2000)

## Filter (multiply spectrogram) with the uneven triangle

In [191]:
INXS_H = INXS * H

In [ ]:
fig = plt.figure(figsize=(16,6))
plt.pcolormesh(t1, f1, 20*np.log10(np.abs(INXS_H)))
plt.ylabel('Frequency (Hz)')
plt.xlabel('Time (sec)')

# Adjust this to show your filter range
plt.ylim(0,2000)

# When ready, save your figure
# fig.savefig(ClassPath + 'filtered-spec/' + username + '.png')

## Use Librosa *ISTFT* to resynthesize sound

In [ ]:
inxs_h = librosa.istft(INXS_H)
ipd.Audio(inxs_h, rate=fs48)

# When you're ready, save the sound to a wav file
# sf.write(ClassPath + 'filtered-sound/' + username + '-filtered.wav', inxs_h, fs48)

## Identify some frequency peaks in your filtered band

In [ ]:
INXS_Hp = np.zeros((1025,2106) )

nPeaks = 1

for n in range(nFrames):
  sort_idx = np.argsort( np.abs(INXS_H[:,n]) )
  peak_idxs = sort_idx[-nPeaks:len(sort_idx)]
  INXS_Hp[peak_idxs, n] = INXS_H[peak_idxs, n]

plt.pcolormesh(t1, f1, 20*np.log10(np.abs(INXS_Hp)))
plt.ylabel('Frequency (Hz)')
plt.xlabel('Time (sec)')
plt.ylim(0,2500)

In [ ]:
inxs_hp = librosa.istft(INXS_Hp)
plt.plot(inxs_hp)
ipd.Audio(inxs_hp, rate=fs48)

# my_inxs_maxf = ...
# sf.write(ClassPath + 'peak-sound/' + username + '-%d.wav' % nPeaks, inxs_hp, fs48)

# Compute the per frame energy in our filtered band

In [ ]:
hn = np.sum(np.abs(INXS_H - INXS_Hp), axis=0) / 1000
np.shape(hn)
plt.plot(hn)

## Generate and filter white noise

In [74]:
noiz = 2*np.random.rand(len(inxs)) - 1

In [ ]:
NOIZ = librosa.stft(noiz)

plt.pcolormesh(t1, f1, 20*np.log10(np.abs(NOIZ)))
plt.ylabel('Frequency (Hz)')
plt.xlabel('Time (sec)')

## Shape (multiply) the noise by the energy in our filter band

In [ ]:
hn.shape = (1,2106)
HN = np.tile(hn, (1025,1))

H_env = H * HN
plt.pcolormesh(t1, f1, H_env)
plt.ylabel('Frequency (Hz)')
plt.xlabel('Time (sec)')

plt.ylim(0,5000)


In [ ]:
INXS_HN_env = NOIZ * H_env
plt.pcolormesh(t1, f1, 20*np.log10(np.abs(INXS_HN_env)))
plt.ylabel('Frequency (Hz)')
plt.xlabel('Time (sec)')

# Adjust for your filter
plt.ylim(0,5000)

# Save plot, when you're ready
# plt.savefig(ClassPath + 'noise-filter/' + username + '.png')

In [ ]:
inxs_noiz_env = librosa.istft(INXS_HN_env)
plt.plot(inxs_noiz_env)
ipd.Audio(inxs_noiz_env,rate=fs48)

# When you're ready, save the sound to a wav file
# sf.write(ClassPath + 'noise-sound/' + username + '.wav', inxs_noiz_env, fs48)

# Add the shaped noise plus max frequency peak in the filtered band

In [ ]:
inxs_peaks_plus_noise = inxs_noiz_env + 2*inxs_hp
plt.plot(inxs_peaks_plus_noise)
ipd.Audio(inxs_peaks_plus_noise, rate=fs48)

# When you're ready, save the sound to a wav file
# sf.write(ClassPath + 'noise-sound/' + username + '-peaks+noise.wav', inxs_peaks_plus_noise, fs48)

# Computing Compression

In [ ]:
fs = 44100
bits = 16
channels = 2

bitrate = fs * bits * channels
bitrate

1411200

## Fourier Transform: Is it a bad deal?

In [129]:
nSamples = 1024
nBits = 16

nSamples * nBits

16384

In [130]:
type(inxs[0])

numpy.float64

In [131]:
type(INXS[0,0])

numpy.complex128

In [ ]:
nFFT = 1024
nComplex = 2
nBits = 16

In [153]:
import pickle

In [169]:
# Size in bytes
len(inxs)*2

2156114

In [168]:
type(inxs[0])

numpy.float64

In [170]:
P = pickle.dumps(np.int16(inxs))
len(P)

2156273

In [ ]:
INXS_64 = np.complex64(INXS)
inxs_64 = librosa.istft(INXS_64)
ipd.Audio(inxs_64, rate=fs48)

In [ ]:
INXS_q8 = np.round(INXS*128/350)
plt.plot(np.max(np.imag(INXS_q8), axis=0))
INXS_q8[:10]

In [ ]:
INXS[100:110]

In [ ]:
inxs_q8 = librosa.istft(INXS_q8)
ipd.Audio(inxs_q8, rate=fs48)